<a href="https://colab.research.google.com/github/geee05/Fine-Tuning-hugging-face-models/blob/main/destroboman_distil_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data

In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [3]:
notebook_login()

In [4]:
model_id = "destroboman/my-depression-model"
##dataset_id = "ag_news"
# replace the value with your model: ex <hugging-face-user>/<model-name>
repository_id = "geee05/destroboman"

In [5]:
import pandas as pd
from datasets import ClassLabel, Dataset
data=pd.read_csv('/content/DAIC_train_3sp_sampled.csv')
test=pd.read_csv('/content/DAIC_test_3sp.csv')
#label_test = ClassLabel(num_classes=2, names=[0, 1])

data.rename(columns = {'Group':'label'}, inplace = True)
test.rename(columns = {'Group':'label'}, inplace = True)

dataset = Dataset.from_dict(data)
test_dataset=Dataset.from_dict(test)

dataset = dataset.class_encode_column("label")
test_dataset = test_dataset.class_encode_column("label")

Stringifying the column:   0%|          | 0/10264 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10264 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/1914 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [6]:
data['label'].value_counts()

0    5132
1    5132
Name: label, dtype: int64

In [7]:
type(dataset)

datasets.arrow_dataset.Dataset

In [8]:

# Training and testing datasets
train_dataset = dataset
test_dataset = test_dataset
# Validation dataset
##val_dataset = dataset['test'].shard(num_shards=2, index=1)

# Preprocessing
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# This function tokenizes the input text using the RoBERTa tokenizer.
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    #print(tokenizer(batch["text"], padding=True, truncation=True, max_length=256)['input_ids'])
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
##val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Map:   0%|          | 0/10264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [9]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [10]:
# Set dataset format
train_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
#val_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
test_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])

In [11]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [12]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features['label'].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

number of labels: 2
the labels: ['0', '1']


In [13]:
# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [14]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate



   learning_rate=2e-5,


In [16]:
import numpy as np
# Model
model = AutoModelForSequenceClassification.from_pretrained(model_id,ignore_mismatched_sizes=True,from_tf=True, config=config)

# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)
from datasets import load_metric
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.
Cloning https://huggingface.co/geee05/destroboman into local empty directory.


In [17]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.425900,0.770501,0.591432,0.349418
2,0.280200,1.535574,0.653605,0.245734
3,0.004100,2.129355,0.642111,0.241417


<ipython-input-16-844c34a3aef9>:29: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=3849, training_loss=0.42061874757272355, metrics={'train_runtime': 827.1273, 'train_samples_per_second': 37.228, 'train_steps_per_second': 4.653, 'total_flos': 2023534725413184.0, 'train_loss': 0.42061874757272355, 'epoch': 3.0})